# Data Aquisition

## Loading Data and Libraries

In [1]:
import pycep_correios as pycep
import geocoder
import pandas as pd
import csv
import json
import os

DATAPATH = '../data/'

In [2]:
df_ceps = pd.read_csv(os.path.join(DATAPATH, 'raw', 'preprocessed', 'CEPs.csv'), header=None)
df_ceps = df_ceps[0]

## Informações básicas

### Amostra

In [3]:
df_ceps.sample(5)

29264    13475250
18827    28380000
1767     65081784
2064     39330000
16372    54325660
Name: 0, dtype: int64

### Shape dos dados

In [4]:
df_ceps.shape

(30142,)

## Pré-processamento

Os CEPs estão no formato Int, com isso o 0 inicial é descartado, o que não é verdade para a lógica real do CEP.

In [5]:
df_ceps.apply(lambda x: len(str(x))).value_counts()

8    25809
7     4333
Name: 0, dtype: int64

Transformação para String e adição do zero à esquerda, quando aplicável.

In [6]:
df_ceps = df_ceps.astype(str).apply(lambda x: x.zfill(8))

In [7]:
df_ceps.apply(lambda x: len(str(x))).value_counts()

8    30142
Name: 0, dtype: int64

## Obter localização (lat, long) a partir do CEP

### Consultar Endereço a partir do CEP através da API dos Correios

In [96]:
def get_address(cep):
    address = None
    try:
        address = pycep.consultar_cep(cep)
    except:
        pass
    return address

In [97]:
cep_address_map = {}

In [98]:
ceps = df_ceps.to_list()

In [251]:
i = 0
for cep in ceps:
    if i % 100 == 0: print(i)
    cep_address_map[cep] = get_address(cep)
    i += 1

In [102]:
cep_address_map_2 = {}
for i in cep_address_map:
    cep = cep_address_map[i]
    if cep != None:
        cep_address_map_2[i] = cep

In [103]:
df_address = pd.DataFrame(cep_address_map_2.values(), index=cep_address_map_2.keys())

In [252]:
df_address.head()

In [106]:
df_address.to_csv('../data/raw/address.csv')

### Consultar a localização a partir do endereço através do Open Street Map

In [253]:
cep_location_map = {}

j = 0
for i, row in data[data['lat'].isnull()].iterrows():
    if j % 100 == 0: print(j)
    
    cep_location_map[i] = {'lng': None, 'lat': None}
    
    if row['end'] and row['uf']:
    
        geo = geocoder.osm(row['end'] + ', ' + row['uf'] + ', Brasil')

        if geo.ok:
            cep_location_map[i] = {'lng': geo.osm['x'], 'lat': geo.osm['y']}
    j += 1

In [145]:
df_locations = pd.DataFrame(cep_location_map.values(), index=cep_location_map.keys())

In [254]:
df_locations.shape

In [255]:
df_locations['lat'].isnull().sum() / df_locations.shape[0]

In [148]:
df_locations.to_csv('../data/raw/locations.csv')

## Juntar informações

In [71]:
data = pd.merge(df_locations, df_address, left_index=True, right_index=True, how='inner')

In [72]:
data.head()

,lat,lng,bairro,cep,cidade,complemento2,end,uf,unidadesPostagem
69900970,-9.970812,-67.791155,Centro,69900970,Rio Branco,None,Avenida Epaminondas Jácome 2858,AC,[]
69945000,NaN,NaN,None,69945000,Acrelândia,None,None,AC,[]
69901024,-9.964670,-67.799725,Conjunto Jardim São Francisco,69901024,Rio Branco,None,Rua Laranja,AC,[]
69850000,NaN,NaN,None,69850000,Boca do Acre,None,None,AM,[]
69930000,NaN,NaN,None,69930000,Xapuri,None,None,AC,[]


## Exportar dados

In [250]:
data.to_csv('../data/external/cep_location.csv')